# DuRecDial Data 이쁘게출력
## UniMIND가 Seeker와 User을 구분하고 모델링하는지 Code 체크하기 (model, data_reader.py 체크)

In [1]:
import json, sys, os, pickle
import pandas as pd

In [164]:
pd.Series(data={'이름':['김한양','정나라','송서울','서성동'],'학년':[3,1,2,2],'점수':[40,45,30,48]})

이름    [김한양, 정나라, 송서울, 서성동]
학년            [3, 1, 2, 2]
점수        [40, 45, 30, 48]
dtype: object

In [211]:
df=pd.DataFrame({'이름':['김한양','정나라','송서울','서성동'], '학년':[3,1,2,2], '점수':[40,45,30,48]}, index=['std1','std2','std3','std4'])
df['전공']=['경영학부', '무용학과', '영어영문학', '신소재공학과']
# df=df['score']+50
df.rename(index={0:'std1', 1:'std2', 2:'std3', 3:'std4'}, inplace=True)
df.rename(columns={'이름':'name', '학년':'year', '점수':'score'}, inplace=True)
# df.rename(index={'std3':'학생3'}, columns={'year':'학년'}, inplace= True)
df.iloc[2,[:]]

SyntaxError: invalid syntax (2937113177.py, line 7)

In [142]:
df.iloc[2:3]

,이름,학년,점수,전공
std3,송서울,2,30,영어영문학


In [115]:
import jsonlines
datafile='data/durecdial/{}.jsonl'
v2datafile='data/durecdial/durec_data_v2/en_{}.txt'
v1linesTrain=[]
v1linesTest=[]
v1linesDev=[]
v2linesTest=[]
v2linesTrain=[]
v2linesValid=[]

with jsonlines.open(datafile.format("train")) as f:
    for line in f.iter(): v1linesTrain.append(line)
with jsonlines.open(datafile.format("test")) as f:
    for line in f.iter(): v1linesTest.append(line)
with jsonlines.open(datafile.format("dev")) as f:
    for line in f.iter(): v1linesDev.append(line)
with jsonlines.open(v2datafile.format("test")) as f:
    for line in f.iter(): v2linesTest.append(line)
with jsonlines.open(v2datafile.format("train")) as f:
    for line in f.iter(): v2linesTrain.append(line)
with jsonlines.open(v2datafile.format("dev")) as f:
    for line in f.iter(): v2linesValid.append(line)
v1linesTrain[0].keys(), v2linesTest[0].keys() , len(v1linesTrain) , len(v2linesTest) , len(v2linesTrain), len(v2linesValid)

(dict_keys(['knowledge', 'conversation']),
 dict_keys(['goal', 'user_profile', 'conversation', 'goal_topic_list', 'goal_type_list', 'situation', 'knowledge']),
 6618,
 1752,
 5678,
 811)

| data  | UniV1 | V2_en | V2_zh(V1) |
|-------|-------|-------|-----------|
| train | 6618  | 5678  | 5678      |
| test  | 2626  | 1752  | 1752      |
| dev   | 946   | 811   | 811       |

In [106]:
v1linesTrain[0]['knowledge']['item_history']

[0, 1, 2, 3, 4, 5]

In [34]:
v1linesTrain[0]['conversation']

[{'role': 'user', 'utterance': '周迅的星座是什么？'},
 {'role': 'system',
  'goal': '问答',
  'utterance': '周迅是天秤座。',
  'knowledge': ['周迅'],
  'item': [],
  'item_id': [],
  'know_text': ['周迅，主演，李米的猜想',
   '周迅，获奖，中国电影金鸡奖_最佳女主角、香港电影金像奖_金像奖-最佳原创电影歌曲奖、台湾电影金马奖_金马奖-最佳女主角奖、亚洲电影大奖_最佳女演员、华语电影传媒大奖_最佳女演员',
   '周迅，出生地，中国浙江衢州',
   '周迅，体重，41kg',
   '周迅，成就，香港电影金像奖最佳女主角、亚洲电影大奖最佳女主角、香港电影金紫荆奖最佳女主角、台湾电影金马奖最佳女主角、中国电视金鹰奖最受欢迎女演员',
   '周迅，评论，就是喜欢你。演技超好，脸蛋真漂漂。、迄今为止演艺圈最有灵性的内地女演员、#印象周迅#好演员，支持！！！！、最喜欢有独特气质的人了、她是真正的为演艺而生。前无借鉴，后无来者。',
   '周迅，生日，1974-10-18',
   '周迅，身高，161cm',
   '周迅，星座，天秤座',
   '周迅，血型，O型']},
 {'role': 'user', 'utterance': '你真棒！知道的真多。'},
 {'role': 'system',
  'goal': '关于明星的聊天',
  'utterance': '我还知道周迅可是亚洲电影大奖最佳女主角呢！',
  'knowledge': ['周迅'],
  'item': [],
  'item_id': [],
  'know_text': ['周迅，主演，李米的猜想',
   '周迅，获奖，中国电影金鸡奖_最佳女主角、香港电影金像奖_金像奖-最佳原创电影歌曲奖、台湾电影金马奖_金马奖-最佳女主角奖、亚洲电影大奖_最佳女演员、华语电影传媒大奖_最佳女演员',
   '周迅，出生地，中国浙江衢州',
   '周迅，体重，41kg',
   '周迅，成就，香港电影金像奖最佳女主角、亚洲电影大奖最佳女主角、香港电影金紫荆奖最佳女主角、台湾电影金马奖最佳女主角、中国电视金鹰奖最受欢迎女演员',
 

In [137]:
def v1RoleCheck(vlines):
    def v1RoleCheckTF(i,conv_dic):
        convs = conv_dic.get('conversation')
        # print(convs)
        if convs[0].get('role') == 'user':
            return 1
        else:
            return 0
    userStart, sysStart=[],[]
    for idx,v in enumerate(vlines):
        if v1RoleCheckTF(idx,v): userStart.append(idx)
        else: sysStart.append(idx)
    print((len(userStart), len(sysStart)))
    return userStart, sysStart
v1RoleCheck(v1linesTrain)

(1570, 1056)


([0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  10,
  11,
  12,
  15,
  16,
  17,
  18,
  20,
  21,
  23,
  24,
  25,
  27,
  31,
  32,
  34,
  37,
  38,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  57,
  58,
  60,
  62,
  63,
  64,
  65,
  66,
  69,
  71,
  73,
  74,
  75,
  76,
  77,
  78,
  80,
  81,
  84,
  87,
  88,
  89,
  91,
  93,
  94,
  95,
  96,
  98,
  99,
  102,
  103,
  105,
  106,
  107,
  110,
  111,
  112,
  113,
  114,
  115,
  117,
  118,
  122,
  124,
  125,
  126,
  128,
  129,
  131,
  133,
  134,
  135,
  139,
  140,
  141,
  142,
  143,
  145,
  146,
  147,
  148,
  149,
  150,
  152,
  153,
  154,
  157,
  158,
  160,
  162,
  164,
  165,
  166,
  168,
  169,
  170,
  171,
  172,
  173,
  174,
  175,
  176,
  177,
  178,
  179,
  180,
  181,
  182,
  183,
  185,
  186,
  187,
  188,
  189,
  190,
  192,
  193,
  194,
  196,
  197,
  198,
  199,
  200,
  201,
  204,
  206,
  208,
  209,
  210,
  212,
  213,
  214,
  216,
  217,

In [4]:
v1lines[9]['conversation'][0]

{'role': 'system',
 'goal': '寒暄',
 'utterance': '李帅洋，最近可好啊',
 'knowledge': ['聊天主题', '聊天时间'],
 'item': [],
 'item_id': [],
 'know_text': ['聊天主题：职位晋升', '聊天时间：晚上22:00，在家里']}

## Du v1 대화 출력하기

In [4]:
def v1conv(convs):
    for conv in convs['conversation']:
        role=conv.get('role').replace('user','USR').replace('system','SYS')
        uttr=conv.get('utterance')
        print(f"{role}: {uttr}")
v1conv(v1linesTrain[8])

SYS: 晚上好啊，金立国，最近学习情况怎么样？
USR: 说起学习我很难受，最近学习退步了。
SYS: 没关系，好好努力，下次会进步的。
USR: 谢谢你的安慰。
SYS: 除了学习以外，闲暇时候也可以看看新闻，你最喜欢谁的新闻呢？
USR: 我最喜欢周杰伦的新闻。
SYS: 正好有条关于周杰伦的新闻说给你听：『台湾歌手周杰伦被聘请成为“中国禁毒宣传形象大使』”。周杰伦他说，他将以阳光健康的形象向广大青少年发出“拒绝毒品，拥有健康”的倡议，并承诺今后将积极宣传毒品危害，倡导全民珍爱生命，远离毒品。
USR: 谢谢你的推荐，我杰伦果然是很正能量的呢。
SYS: 是啊，这样的明星很值得当榜样。
USR: 对呢，我就是因为这些才喜欢他的，满满的正能量。
SYS: 周杰伦有你这样的粉丝也是他的荣幸啊。
USR: 嘿嘿，先不说啦，我再去学习会吧，改天再聊。
SYS: 好的，加油哦，再见。
USR: 再见啦。


In [111]:
print(v1linesTrain[0]['knowledge'].keys())
v1linesTrain[0]['conversation']

dict_keys(['knowledge', 'user_profile', 'item_history'])


[{'role': 'user', 'utterance': '周迅的星座是什么？'},
 {'role': 'system',
  'goal': '问答',
  'utterance': '周迅是天秤座。',
  'knowledge': ['周迅'],
  'item': [],
  'item_id': [],
  'know_text': ['周迅，主演，李米的猜想',
   '周迅，获奖，中国电影金鸡奖_最佳女主角、香港电影金像奖_金像奖-最佳原创电影歌曲奖、台湾电影金马奖_金马奖-最佳女主角奖、亚洲电影大奖_最佳女演员、华语电影传媒大奖_最佳女演员',
   '周迅，出生地，中国浙江衢州',
   '周迅，体重，41kg',
   '周迅，成就，香港电影金像奖最佳女主角、亚洲电影大奖最佳女主角、香港电影金紫荆奖最佳女主角、台湾电影金马奖最佳女主角、中国电视金鹰奖最受欢迎女演员',
   '周迅，评论，就是喜欢你。演技超好，脸蛋真漂漂。、迄今为止演艺圈最有灵性的内地女演员、#印象周迅#好演员，支持！！！！、最喜欢有独特气质的人了、她是真正的为演艺而生。前无借鉴，后无来者。',
   '周迅，生日，1974-10-18',
   '周迅，身高，161cm',
   '周迅，星座，天秤座',
   '周迅，血型，O型']},
 {'role': 'user', 'utterance': '你真棒！知道的真多。'},
 {'role': 'system',
  'goal': '关于明星的聊天',
  'utterance': '我还知道周迅可是亚洲电影大奖最佳女主角呢！',
  'knowledge': ['周迅'],
  'item': [],
  'item_id': [],
  'know_text': ['周迅，主演，李米的猜想',
   '周迅，获奖，中国电影金鸡奖_最佳女主角、香港电影金像奖_金像奖-最佳原创电影歌曲奖、台湾电影金马奖_金马奖-最佳女主角奖、亚洲电影大奖_最佳女演员、华语电影传媒大奖_最佳女演员',
   '周迅，出生地，中国浙江衢州',
   '周迅，体重，41kg',
   '周迅，成就，香港电影金像奖最佳女主角、亚洲电影大奖最佳女主角、香港电影金紫荆奖最佳女主角、台湾电影金马奖最佳女主角、中国电视金鹰奖最受欢迎女演员',
 

In [116]:
from collections import defaultdict
def v1conv(v1line, istopic=True, isgoal=True, iskg=True):
    knows = v1line['knowledge']
    convs = v1line['conversation']
    goallist_nodup=[]
    for conv in convs:
        role = conv['role']
        uttr = conv['utterance']
        if role=='system' :
            if conv['goal'] not in goallist_nodup:
                goallist_nodup.append(conv['goal'])
    return goallist_nodup

v1conv(v1linesTrain[0])

['问日期', '关于明星的聊天', '音乐推荐', '播放音乐', '再见']

## Du 1.0 에서 Goal Sequence 뽑기

In [118]:
def v1conv_goalseq(train,test,dev):
    from collections import defaultdict
    goalSeqDic = defaultdict(int)
    for v1lines in [train,test,dev]:
        for v1line in v1lines:
            goalseq = "__".join(v1conv(v1line))
            goalSeqDic[goalseq]+=1
    return goalSeqDic
v1goalseqdic=v1conv_goalseq(v1linesTrain,v1linesTest,v1linesDev)
v1goalseqdic

defaultdict(int,
            {'问日期__关于明星的聊天__音乐推荐__播放音乐__再见': 344,
             '寒暄__问User姓名__问User性别__问User年龄__问User爱好__再见': 1423,
             '问答__关于明星的聊天__电影推荐__再见': 1340,
             '问天气__美食推荐__兴趣点推荐__再见': 681,
             '问天气__美食推荐__再见': 375,
             '问天气__音乐推荐__播放音乐__再见': 451,
             '寒暄__音乐推荐__关于明星的聊天__新闻推荐__再见': 257,
             '音乐点播__音乐推荐__播放音乐__再见': 582,
             '问日期__关于明星的聊天__电影推荐__再见': 486,
             '寒暄__提问__关于明星的聊天__音乐推荐__播放音乐__再见': 224,
             '问答__关于明星的聊天__音乐推荐__播放音乐__再见': 720,
             '新闻点播__新闻推荐__再见': 425,
             '问时间__天气信息推送__再见': 750,
             '寒暄__提问__关于明星的聊天__电影推荐__再见': 408,
             '寒暄__提问__新闻推荐__再见': 506,
             '寒暄__音乐推荐__关于明星的聊天__电影推荐__再见': 296,
             '寒暄__电影推荐__关于明星的聊天__音乐推荐__播放音乐': 8,
             '寒暄__电影推荐__关于明星的聊天__音乐推荐__播放音乐__再见': 314,
             '问日期__关于明星的聊天__音乐推荐__播放音乐': 8,
             '寒暄__提问__关于明星的聊天__音乐推荐__播放音乐': 16,
             '寒暄__新闻推荐__关于明星的聊天__电影推荐__再见': 47,
             '问答__

In [122]:
## 저장
import pandas as pd
pd.DataFrame(v1goalseqdic, index=['count']).T.to_csv("./total-DuRec_1_GoalSeq.csv",encoding='utf-8')

# DuRecDial V2.0

In [5]:
def v2conv(conv, istopic=True, isgoal=True, iskg=True):
    """
    Args:
        conv: v2lines[i]
        istopic: text Topic 출력여부
        isgoal: text type(goal)출력여부
        iskg: text kg 출력여부
    Returns: [uttr lines], {'uttrs':uttrs, 'topics':topics, 'goals':goals, 'kgs':kgs,'roles':roles}

    """
    topics = conv.get('goal_topic_list') if istopic else ["" for _ in range(len(conv.get('goal_type_list')))]
    goals = conv.get('goal_type_list') if isgoal else ["" for _ in range(len(conv.get('goal_type_list')))]
    kgs = conv.get('knowledge')# if iskg else ["" for _ in range(len(conv.get('goal_type_list')))]
    uttrs = conv.get('conversation')
    # if uttrs[0]=='[': uttrs = uttrs[3:]
    roles=["SYS","USR"] if conv.get('user_profile').get('Name').lower() in uttrs[0].lower() else ["USR","SYS"]
    lines=[]

    for i in range(len(topics)):
        line = f"{roles[i%2]}: {uttrs[i] if uttrs[i][0]!='[' else uttrs[i][4:]}{' , Goal_Topic: '+topics[i] if istopic else ''}{', Goal_Type: '+goals[i] if isgoal else ''}{', KG: '+ ', '.join(kgs[i]) if iskg else ''}"
        lines.append(line)
    return lines, {'uttrs':uttrs, 'topics':topics, 'goals':goals, 'kgs':kgs,'roles':roles}
def v2conv_mktxt(convs):
    with open("./V2ConvSave.txt",'w',encoding='utf-8') as f:
        for i,conv in enumerate(convs):
            lines=v2conv(conv)
            f.write(f"ConvIdx: {i}\n")
            for line in lines:
                f.write(line)
                f.write('\n')
            f.write('\n')

# v2conv_mktxt(v2lines)
v2conv(v2linesTest[0], istopic=True, isgoal=False)

(["USR: What is Xun Zhou's star sign? , Goal_Topic: Xun Zhou, KG: Xun Zhou, Star sign, Libra",
  "SYS: It's Libra. , Goal_Topic: Xun Zhou, KG: Xun Zhou, Star sign, Libra",
  'USR: Good for you! You know so much. , Goal_Topic: Xun Zhou, KG: ',
  'SYS: I also know that she has won the Asian Film Awards for Best Actress. , Goal_Topic: Xun Zhou, KG: Xun Zhou, Awards, Asian Film Awards _ Best Actress',
  "USR: She's my idol. Her acting skills are excellent. , Goal_Topic: Xun Zhou, KG: Xun Zhou, Comments, I am a big fan! Amazing acting skills and gorgeous look!",
  "SYS: She was born for acting and has won the Chinese Film Media Awards for Best Actress. , Goal_Topic: Xun Zhou, KG: Xun Zhou, Comments, She was really born for acting. There's no reference before, nor after.",
  'USR: I like watching movies by her very much. , Goal_Topic: Xun Zhou, KG: ',
  'SYS: Here\'s a movie for you. It\'s called The Equation of Love & Death starred by her. In this movie, she devoted rich passion and sinceri

In [138]:
## V2 Goal list 출력확인
# for conv in v2lines:
# v2lines[0]['goal_type_list']
v2goalset=set()
for idx,i in enumerate(v2linesTrain[:29]):
    for g in i['goal_type_list']:
        if g=='Ask questions':
            pass
            # print(idx, i)
        v2goalset.add(g)
# v2goalset, \
len(v2goalset)

23 {'goal': '[1]Greetings-->[2]Ask questions-->[3] Ask questions(Once a Thief)-->[4] Chat about stars(Leslie Cheung)-->[5] Movie recommendation(Buenos Aires Zero Degree、Farewell My Concubine)-->[6]Say goodbye', 'user_profile': {'Age Range': '18-25', 'Name': 'Xinshan Niu', 'Residence': 'Rizhao', 'Accepted food': 'poached spicy slices of pork', 'Accepted movies': [], 'Accepted Music': [], 'Rejected music': [], 'Gender': 'Female', 'Accepted celebrities': ['Leslie Cheung', 'Jacky Cheung'], 'Accepted movie': ['Once a Thief'], 'Reject': ['Poi'], 'Rejected movies': [], 'Favorite news': ["Leslie Cheung's news", "Jacky Cheung's news"], 'Accepted music': 'Wolf Legend', 'Occupation': 'Student'}, 'conversation': ["[1] Hello, Ms.Xinshan Niu. How's it going with your study?", "Don't mention it. I've degressed. I will be scolded again.", "It's OK. Just study harder and relax yourself.", 'Yes, I have to think positively and move on.', "[2] Don't put too much pressure on yourself. Let's talk about movi

19

In [127]:
v2conv(v2linesTrain[26], isgoal=True, istopic=False, iskg=False)

['USR: Hi, do you know what the weather is like today?, Goal_Type: Ask about weather',
 "SYS: It's sunny with north wind. The highest temperature will be 26℃, and the lowest temperature will be 19℃., Goal_Type: Ask about weather",
 "USR: You're so smart. Thank you., Goal_Type: Ask about weather",
 "SYS: I wanna recommend Stir-Fried Venerupis to you. It's suitable for eating it in such weather., Goal_Type: Food recommendation",
 "USR: OK, I do want to eat it. I haven't eaten it for a long time., Goal_Type: Food recommendation",
 'SYS: You can try it today., Goal_Type: Food recommendation',
 'USR: Mm-hmm, the fresh venerupis with some scallions taste great!, Goal_Type: Food recommendation',
 'SYS: The Stir-Fried Venerupis at Wanzhou Roasted Fish is tasty. You can also order it at this store., Goal_Type: POI recommendation',
 "USR: How much does it cost for one person at this store? And how's the score?, Goal_Type: POI recommendation",
 'SYS: It costs 48 yuan for one person with a score o

# DuRec 에서 사용자가 추천해달라고 했는데, SYSTEM이 추천하지않고 오히려 질문한 경우가 있을까?

In [65]:
def v2conv_usersRec(dd,convs):
    lines, convdic = v2conv(convs) # return lines, {'uttrs':uttrs, 'topics':topics, 'goals':goals, 'kgs':kgs,'roles':roles}
    for idx in range(len(convdic['uttrs'])):
        role=convdic['roles'][idx%2]
        if role=='USR' and idx<len(convdic['uttrs'])-1 and 'recommendation' in convdic['uttrs'][idx] and 'thank' not in convdic['uttrs'][idx].lower():
            nextSysGoal=convdic['goals'][idx+1].lower()
            if not list(filter(lambda x:x in convdic['uttrs'][idx+1] , convdic['kgs'][idx+1])) and "goodbye" not in nextSysGoal:
                print(dd,idx,"FIND", convdic['uttrs'][idx])

_=[[idx,v2conv_usersRec(idx,i)] for idx,i in enumerate(v2linesTrain)]
# v2conv_usersRec(578, v2lines[578])


181 6 FIND Okay, I'll take your recommendation.
308 4 FIND Good recommendation.
1210 10 FIND I'm sure your recommendation is correct. I'll watch it later.
1350 4 FIND I haven't eaten it for a long time. Good recommendation.
1379 11 FIND Sorry, I'm not interested in this song. Any other recommendations?
1507 15 FIND I want to listen to it hearing your recommendation.
1665 19 FIND Your recommendation is really good.
1736 11 FIND I've heard this song. Any other recommendations?


In [75]:
v2conv(v2linesTrain[1665])[0][18:]

['SYS: OK, here it goes. , Goal_Topic: Where Are We Going, Dad?\xa0, Goal_Type: Play music, KG: ',
 'USR: Your recommendation is really good. , Goal_Topic: Where Are We Going, Dad?\xa0, Goal_Type: Play music, KG: ',
 "SYS: I'm glad you like it. , Goal_Topic: Where Are We Going, Dad?\xa0, Goal_Type: Play music, KG: ",
 "USR: It's getting late. I'm going to bed. Bye. , Goal_Topic: Say goodbye, Goal_Type: Say goodbye, KG: ",
 'SYS: Bye! , Goal_Topic: Say goodbye, Goal_Type: Say goodbye, KG: ',
 'USR: Bye! , Goal_Topic: Say goodbye, Goal_Type: Say goodbye, KG: ']

## Goal Sequence Check 분석

### Goal 개수 파악

In [64]:
v2linesValid[0]['goal_type_list']

['Ask about weather',
 'Ask about weather',
 'Ask about weather',
 'Food recommendation',
 'Food recommendation',
 'Food recommendation',
 'Food recommendation',
 'POI recommendation',
 'POI recommendation',
 'POI recommendation',
 'POI recommendation',
 'POI recommendation',
 'POI recommendation',
 'POI recommendation',
 'POI recommendation',
 'POI recommendation',
 'Say goodbye',
 'Say goodbye']

In [67]:
def v2convGoalTotal(train, test, valid):
    counter=defaultdict(int)
    for v2lines in [train,test,valid]:
        for v2line in v2lines:
            for goal in v2line['goal_type_list']:
                 counter[goal]+=1
    return counter
v2convGoalTotalDic=v2convGoalTotal(v2linesTrain,v2linesTest,v2linesValid)
len(v2convGoalTotalDic.keys()), v2convGoalTotalDic

(19,
 defaultdict(int,
             {'Q&A': 6075,
              'Chat about stars': 16280,
              'Movie recommendation': 14882,
              'Say goodbye': 17833,
              'Ask about weather': 4393,
              'Food recommendation': 4465,
              'POI recommendation': 5451,
              'Music on demand': 1692,
              'Music recommendation': 13174,
              'Play music': 10037,
              'Greetings': 10666,
              "Ask about user's name": 2850,
              "Ask about user's gender": 2860,
              "Ask about user's age": 2850,
              "Ask about user's hobbies": 5006,
              'Ask about date': 2401,
              'Ask questions': 2100,
              'Ask about time': 2221,
              'Weather notification': 2437}))

전체 골의 (동시에 골2~3개 포함)개수는 19개 이다.

'Q&A': 6075,
'Chat about stars': 16280,
'Movie recommendation': 14882,
'Say goodbye': 17833,
'Ask about weather': 4393,
'Food recommendation': 4465,
'POI recommendation': 5451,
'Music on demand': 1692,
'Music recommendation': 13174,
'Play music': 10037,
'Greetings': 10666,
"Ask about user's name": 2850,
"Ask about user's gender": 2860,
"Ask about user's age": 2850,
"Ask about user's hobbies": 5006,
'Ask about date': 2401,
'Ask questions': 2100,
'Ask about time': 2221,
'Weather notification': 2437

In [183]:
## Goal Sequence Check
def v2conv_usersRec(dd,convs):

    lines, convdic = v2conv(convs) # return lines, {'uttrs':uttrs, 'topics':topics, 'goals':goals, 'kgs':kgs,'roles':roles}
    goalset=[]
    for goal in convdic['goals']:
        if goal.lower() in goalset: pass
        else: goalset.append(goal.lower())

    goalString=""
    goalStringlower=""
    for goal in convdic['goals']:
        if goal in goalString: pass
        else:
            goalString+=f'{goal}__'
            goalStringlower+=f'{goal.lower()}__'
    if goalString=="Greetings__Ask about user's name__Ask about user's gender__Ask about user's age__Ask about user's hobbies__Say goodbye__":
        print(dd)
    return goalString[:-2], goalStringlower[:-2]
v2convgoalSeq_train=[v2conv_usersRec(idx,i)[0] for idx,i in enumerate(v2linesTrain)]
v2convgoalSeq_test=[v2conv_usersRec(idx,i)[0] for idx,i in enumerate(v2linesTest)]

4
12
16
24
25
27
28
29
31
42
43
50
51
62
65
66
77
78
82
88
89
91
93
99
111
113
114
117
118
124
132
133
137
138
148
154
165
182
185
186
187
188
189
196
205
212
215
216
224
235
237
238
243
246
266
268
276
278
285
289
292
296
298
316
322
325
329
336
347
351
357
369
372
375
376
380
384
394
397
405
406
408
409
410
418
419
422
429
432
446
447
454
459
468
477
488
493
495
497
498
500
504
518
522
528
540
559
573
576
577
583
589
591
592
593
617
620
621
624
625
638
641
643
645
652
658
682
687
689
693
699
703
705
723
724
730
739
753
757
759
772
780
785
791
792
795
796
799
804
806
809
813
817
822
827
837
846
851
853
854
856
857
858
862
872
877
880
891
893
896
899
901
902
919
925
929
932
933
944
962
963
965
975
980
981
982
994
999
1000
1006
1013
1018
1031
1032
1034
1035
1036
1037
1041
1045
1048
1060
1063
1075
1082
1083
1092
1105
1110
1113
1130
1131
1133
1146
1149
1156
1159
1162
1165
1170
1176
1191
1194
1202
1205
1209
1210
1214
1217
1220
1232
1234
1235
1236
1239
1241
1244
1254
1257
1266
1267
1271
129

In [169]:
v2convgoaldic_train=dict()
v2convgoaldic_test=dict()
for goalseq in v2convgoalSeq_train:
    if goalseq in v2convgoaldic_train:
        v2convgoaldic_train[goalseq]+=1
    else:
        v2convgoaldic_train[goalseq]=1
for goalseq in v2convgoalSeq_test:
    if goalseq in v2convgoaldic_test:
        v2convgoaldic_test[goalseq]+=1
    else:
        v2convgoaldic_test[goalseq]=1
print("Train",sum(list(map(lambda x : x[1],v2convgoaldic_train.items()))), len(v2convgoalSeq_train))
print("Test",sum(list(map(lambda x : x[1],v2convgoaldic_test.items()))), len(v2convgoalSeq_test))

Train 5678 5678
Test 1752 1752


In [172]:
sorted(v2convgoaldic_test.items())

[('Ask about date__Chat about stars__Movie recommendation__Music recommendation__Play music__Say goodbye',
  2),
 ('Ask about date__Chat about stars__Movie recommendation__Say goodbye', 84),
 ('Ask about date__Chat about stars__Music recommendation__Play music__Say goodbye',
  59),
 ('Ask about time__Weather notification__Say goodbye', 154),
 ('Ask about weather__Food recommendation__POI recommendation__Say goodbye',
  130),
 ('Ask about weather__Food recommendation__Say goodbye', 81),
 ('Ask about weather__Music recommendation__Play music__Say goodbye', 80),
 ("Greetings__Ask about user's name__Ask about user's gender__Ask about user's age__Ask about user's hobbies__Say goodbye",
  270),
 ('Greetings__Ask questions__Chat about stars__Movie recommendation__Say goodbye',
  87),
 ('Greetings__Ask questions__Chat about stars__Music recommendation__Play music__Say goodbye',
  58),
 ('Greetings__Movie recommendation__Chat about stars__Music recommendation__Play music__Say goodbye',
  133),


## Topic 개수

In [68]:
from collections import defaultdict
def v2convTopicsTotal(train,test,valid):
    alltopics=defaultdict(int)
    for v2lines in [train,test,valid]:
        for v2line in v2lines:
            _, convdic = v2conv(v2line)
            for topic in convdic['topics']:
                alltopics[topic]+=1
    print("전체 토픽 종류카운팅", len(alltopics.keys()))
    print("전체 토픽 카운팅",sum(alltopics.values()))
    return alltopics
v2convTopicsTotal(v2linesTrain,v2linesTest,v2linesValid)

전체 토픽 종류카운팅 646
전체 토픽 카운팅 127673


defaultdict(int,
            {'Left Right Love Destiny': 297,
             'Cecilia Cheung': 855,
             'Failan': 55,
             'The Stool Pigeon': 108,
             'Say goodbye': 17813,
             'Ask about weather': 4392,
             'Marinated Fish': 797,
             'Mr.Fish Roasted': 78,
             'Hand in Hand': 66,
             'Change Me': 56,
             'All the Things You Never Knew': 322,
             'Sautéed\xa0Spicy\xa0Pork\xa0': 46,
             'Laoqi Sichuan Restaurant': 115,
             'Greetings': 10666,
             "Ask about user's name": 2850,
             "Ask about user's gender": 2860,
             "Ask about user's age": 2850,
             "Ask about user's hobbies": 5006,
             'Ask about date': 2401,
             'Andy Lau': 636,
             'A World Without Thieves': 89,
             'Sliced\xa0Fish\xa0in\xa0Hot\xa0Chili\xa0Oil': 208,
             'Ruby Lin': 563,
             'You Deserve To Be Single': 31,
             'Nau

## Topic Pattern

In [58]:
## Topic - Goal Pair
from collections import defaultdict
def _v2convGTPair(v2line):
    _, convdic = v2conv(v2line)
    txts=[]
    for g,t in (zip(convdic['goals'], convdic['topics'])):
        txts.append(f'{g}__{t}')
    return txts
def v2convGTPair(v2lines):
    counter=defaultdict(int)
    for v2line in v2lines:
        gt_texts = _v2convGTPair(v2line)
        for gt in gt_texts:
            counter[gt]+=1
    return counter
def v2convGTPairTotal(train, test, valid):
    counter=defaultdict(int)
    for v2lines in [train,test,valid]:
        for v2line in v2lines:
            gt_texts = _v2convGTPair(v2line)
            for gt in gt_texts:
                counter[gt]+=1
    return counter
v2trainGTdic=v2convGTPair(v2linesTrain)
v2testGTdic=v2convGTPair(v2linesTest)
v2validGTdic=v2convGTPair(v2linesValid)
v2totalGTdic=v2convGTPairTotal(v2linesTrain,v2linesTest,v2linesValid)

In [60]:
sum(v2trainGTdic.values()), sum(v2testGTdic.values()), sum(v2validGTdic.values()),sum(v2totalGTdic.values())

(87301, 27700, 12672, 127673)

In [42]:
len(v2trainGTdic.keys()),len(v2testGTdic.keys())

(974, 790)

In [50]:
for k,v in sorted(v2trainGTdic.items(), key= lambda item : item[1], reverse=True):
    # print(k)
    if "ask" in k.lower() and "hazy later" in k.lower():
        print(k,v)

Ask about weather__Sunny early and hazy later. Variable winds. The high will be - 3 ℃, and low - 12 ℃ 1


In [20]:
sorted(v2testGTdic.items(), key= lambda item : item[1], reverse=True)

[('Say goodbye--Say goodbye', 3800),
 ('Greetings--Greetings', 2504),
 ("Ask about user's hobbies--Ask about user's hobbies", 945),
 ('Ask about weather--Ask about weather', 873),
 ("Ask about user's gender--Ask about user's gender", 544),
 ("Ask about user's age--Ask about user's age", 542),
 ("Ask about user's name--Ask about user's name", 540),
 ('Weather notification--Weather notification', 512),
 ('Ask questions--Ask questions', 464),
 ('Ask about date--Ask about date', 437),
 ('Ask about time--Ask about time', 434),
 ('Chat about stars--Leslie Cheung', 332),
 ('Chat about stars--Jacky Cheung', 314),
 ('Chat about stars--Jay Chou', 302),
 ('Chat about stars--Leehom Wang', 270),
 ('Chat about stars--Jiong He', 260),
 ('Chat about stars--Kris Wu', 240),
 ('Chat about stars--Xiaoming Huang', 202),
 ('Chat about stars--Xun Zhou', 200),
 ('Chat about stars--Cecilia Cheung', 200),
 ('Food recommendation--Marinated Fish', 183),
 ('Chat about stars--Bo Huang', 160),
 ('Chat about stars--M

In [61]:
import pandas as pd
# pd.DataFrame(v2testGTdic, index=['count']).T.to_csv("./test-DuRectgPair.csv",encoding='utf-8')
# pd.DataFrame(v2trainGTdic, index=['count']).T.to_csv("./train-DuRectgPair.csv",encoding='utf-8')
pd.DataFrame(v2totalGTdic, index=['count']).T.to_csv("./total-DuRectgPair.csv",encoding='utf-8')

___

# DuRecDial
## Data Analysis
1차 목표: UniMIND 에 있는 DuRecDial Dataset 을 확인

### DuRecDial Dataset 만드는 부분

In [1]:
import json, sys, os, pickle
# import torch

In [104]:
import logging
txt='tetete'
tetet=['asdf',123]
# logging.info(f"{txt} {tetet}")
'MRR@10/MRR@50: {}'.format(', '.join(map(str, tetet)))

'MRR@10/MRR@50: asdf, 123'

In [69]:
from transformers import BartForConditionalGeneration, BertTokenizer, BartConfig, AdamW
tokenizer = BertTokenizer.from_pretrained('fnlp/bart-base-chinese', do_lower_case=True, cache_dir='../temp_cache/bart')
tokenizer.add_special_tokens({'additional_special_tokens': ['[goal]', '[user]', '[system]', '[knowledge]', '[item]', '[profile]', '[history]']})

Downloading:   0%|          | 0.00/110k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/449 [00:00<?, ?B/s]

7

In [70]:
data_dict = {'resp':{'source_ids':[], 'target_ids':[], 'item_ids':[]}, 'item':{'source_ids':[], 'target_ids':[], 'item_ids':[]}, 'goal':{'source_ids':[], 'target_ids':[], 'item_ids':[]}, 'know':{'source_ids':[], 'target_ids':[], 'item_ids':[]}}

In [98]:
class DotDict(dict):
    def __init__(self, **kwds):
        self.update(kwds)
        self.__dict__ = self

args = DotDict()
args.do_train, args.do_eval,args.do_pipeline, args.do_finetune, args.overwrite_output_dir, args.overwrite_cache= True, True, True, True, True, True
args.gpu, args.num_train_epochs, args.num_ft_epochs = '0', 1, 1
args.per_gpu_train_batch_size, args.per_gpu_eval_batch_size = 2,2
args.max_seq_length, args.max_target_length=512,100
args.data_name='durecdial'
args.model_name_or_path ='fnlp/bart-base-chinese'
args.output_dir, args.data_dir, args.cache_dir = 'output','data','..\\temp_cache\\bart'
args.use_cached_data=False

In [71]:
import os
# import logging
import pickle
def write_pkl(obj, filename):
    with open(filename, 'wb') as f:
        pickle.dump(obj, f)
def read_pkl(filename):
    with open(filename, 'rb') as f:
        return pickle.load(f)

In [102]:
def load_and_cache_examples(args, tokenizer, evaluate=False):
    mode = 'test' if evaluate else 'train'
    # Load data features from cache or dataset file
    cached_features_file = os.path.join(args.data_dir, 'OLDcached_nl_{}_{}_{}_{}_{}'.format(
        args.data_name, # durecdial
        mode, # train/test
        list(filter(None, args.model_name_or_path.split('/'))).pop(), # bart-base-chinese
        str(args.max_seq_length), # 512
        str(args.max_target_length))) # 100
    # print(cached_features_file)
    # return cached_features_file

    if os.path.exists(cached_features_file) and args.use_cached_data: # 파일있으면, 그대로 pkl load
        # logger.info("Loading features from cached file %s", cached_features_file)
        print(args.use_cached_data)
        features = read_pkl(cached_features_file)
        print("\nLoaded number of instance:", len(features['resp']['source_ids']))
    else: # 없으면 convert_to_features 함수를 통해 파일 생성 & pkl 파일로 저장
        # logger.info("Creating features from dataset file at %s", args.data_dir)
        features = convert_to_features(args, tokenizer, mode)
        print("Loaded number of instance:", len(features['resp']['source_ids']))

        # logger.info("Saving features into cached file %s", cached_features_file)
        write_pkl(features, cached_features_file)
    return features
def merge_dataset(ft_dataset):
    source_ids = []
    target_ids = []
    item_ids = []
    item_dict = ft_dataset['item_dict']
    for task in ['resp','goal','know','item']:
        task_dataset = ft_dataset[task]
        for source_id, target_id, item_id in zip(task_dataset['source_ids'], task_dataset['target_ids'], task_dataset['item_ids']):
            source_ids.append(source_id)
            target_ids.append(target_id)
            item_ids.append(item_id)
    return {'source_ids':source_ids, 'target_ids':target_ids, 'item_ids':item_ids, 'item_dict':item_dict}
def convert_to_features():
    pass
load_and_cache_examples(args,tokenizer)

TypeError: convert_to_features() takes 0 positional arguments but 3 were given

In [ ]:
ft_dataset = load_and_cache_examples(args, tokenizer, evaluate=False)
train_dataset = merge_dataset(ft_dataset)